In [ ]:
import sys
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install transformers==4.37.2

In [ ]:
from transformers import TFBertForMaskedLM, BertTokenizerFast
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MAE
import random

max_seq_length = 20
masking_probability = 0.35  # Probability of masking a word

# Load pre-trained tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = TFBertForMaskedLM.from_pretrained('bert-base-uncased')

# Function to mask random words
def create_masked_input(text):
  input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=max_seq_length, truncation=True, padding='max_length')
  labels = input_ids.copy()

  # Mask random words
  for i in range(len(input_ids)):
    if random.random() < masking_probability:
      if input_ids[i] not in tokenizer.special_tokens_map.values():  # Skip special tokens
        temp = model.config.vocab_size
        if temp == 30522:
          labels[i] = temp-1
        else:
          labels[i] = temp  # Replace with masking label
        input_ids[i] = tokenizer.mask_token_id

  return input_ids, labels

global training_data

kai_messages = ""
training_data = []

with open("/content/_chat.txt", "r") as fin:

    for i in range(1022):
      line = fin.readline()
      if "Ayaan" not in line:
        line = line[26::]
        kai_messages+=line
        input_ids, labels = create_masked_input(kai_messages)
        training_data.append((input_ids, labels))

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers, layers

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

x_train, y_train = zip(*training_data)
x_train = tf.convert_to_tensor(x_train)
y_train = tf.convert_to_tensor(y_train)

# Define model inputs
input_word_ids = Input(shape=(max_seq_length,), dtype=tf.int32)

# Pass through pre-trained model
outputs = model(input_word_ids)[-1]  # Access the masked language modeling output

# Define the model
model_local = Model(inputs=input_word_ids, outputs=outputs)

#tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# Compile the model
model_local.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=Adam(learning_rate=2e-5))

# Train the model
history = model_local.fit(x_train, y_train, epochs=4, batch_size=16)  # Adjust epochs and batch size as needed
history_df = pd.DataFrame(history.history)
history_df["loss"].plot()

# Save the model
model_local.sa
model_local.save("my_mlm_model")
model.config.save_pretrained("/content/my_mlm_model/config.json")

In [ ]:
#call model in this cell

In [ ]:
from transformers import TFBertForMaskedLM, BertTokenizer

# Replace 'bert-base-uncased' with your desired pre-trained model name
model = TFBertForMaskedLM.from_pretrained("/content/my_mlm_model")
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

prompt = "Today is a beautiful day because [MASK] is happening."

# Tokenize the prompt (optional, depending on the model)
encoded_prompt = tokenizer(prompt, return_tensors='tf')
input_ids = encoded_prompt['input_ids']

predictions = model(input_ids)[7]  # Assuming the mask is at index 0
predicted_index = tf.math.argmax(predictions[0]).numpy()
predicted_word = tokenizer.convert_ids_to_tokens(predicted_index)[7]  # Assuming mask is at index 1

print(predicted_word)


In [ ]:
#Test model
from transformers import BertTokenizer
import tensorflow as tf

# Define your prompt
prompt = "My favorite person is [MASK]"

# Load tokenizer (replace with your model name)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Pad the prompt sequence (assuming max_len is 20)
max_len = 20
padded_prompt = tokenizer(prompt, padding='longest', max_length=20, return_tensors='tf')
input_ids = padded_prompt['input_ids']  # Access encoded input IDs

model = TFBertForMaskedLM.from_pretrained("enter path of model")

# Generate predictions
predictions = model(input_ids)[0]
print(predictions)

# Decode the predicted word
#predicted_index = tf.math.argmax(predictions[0][1]).numpy()
predicted_index = tf.math.argmax(predictions[0]).numpy()  # Assuming mask is at index 1
predicted_word = tokenizer.convert_ids_to_tokens(predicted_index)[1]
print(predicted_word)

# Print the completed prompt
completed_prompt = prompt.replace("[MASK]", predicted_word)
print(f"Completed Prompt: {completed_prompt}")
